## System

In [ ]:
%pip install --upgrade pip 
%pip install arcgis

# Run

## Dependencies and Parameters

In [ ]:
import os
import re
import requests

from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import FeatureSet

In [ ]:
input_url = "https://geoelectoral.oep.org.bo/oep/rest/services/"
data_dir = "../data"

In [ ]:
OUTPUT_DIR = os.path.join(data_dir, "")
BASE_URL = input_url

## Functions

In [ ]:
def sanitize(s):
    return re.sub(r'[^\w\-_\.]', '_', s)

In [ ]:
def get_services():
    url = f"{BASE_URL}?f=json"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    services = data.get("services", [])
    return [s for s in services if s["type"] == "MapServer"]


In [ ]:
def get_all_mapservers():
    """
    Returns a list of all MapServer service URLs, including those in folders.
    """
    services = []
    url = f"{BASE_URL}?f=json"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    # Root-level services
    for s in data.get("services", []):
        if s["type"] == "MapServer":
            services.append(f"{BASE_URL}/{s['name']}/{s['type']}")
    # Services in folders
    for folder in data.get("folders", []):
        folder_url = f"{BASE_URL}/{folder}?f=json"
        r2 = requests.get(folder_url)
        r2.raise_for_status()
        folder_data = r2.json()
        for s in folder_data.get("services", []):
            if s["type"] == "MapServer":
                services.append(f"{BASE_URL}/{s['name']}/{s['type']}")
    return services

In [ ]:
# Función para obtener todas las capas de un MapServer
def get_layers(service_url):
    url = f"{service_url}?f=json"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    return data.get("layers", [])


In [ ]:
def download_layer_arcgis(service_url, layer, gis, page_size=1000):
    layer_id = layer["id"]
    layer_name = sanitize(layer["name"])
    full_url = f"{service_url}/{layer_id}"
    service_name = sanitize(service_url.split("/")[-2])
    out_dir = os.path.join(OUTPUT_DIR, service_name)
    os.makedirs(out_dir, exist_ok=True)
    fname = f"{layer_id}_{layer_name}.geojson"
    out_path = os.path.join(out_dir, fname)

    lyr = FeatureLayer(full_url, gis)
    count = lyr.query(where="1=1", return_count_only=True)

    if count > 5000:
        print(f"    ⚠️ {layer_name} tiene demasiados datos ({count} features). Saltando descarga.")
        return

    print(f"    ⏳ Descargando {layer_name} ({count} features)...")

    try:
        # Intenta paginación primero
        features = []
        for offset in range(0, count, page_size):
            q = lyr.query(
                where="1=1",
                out_fields="*",
                return_geometry=True,
                result_offset=offset,
                result_record_count=page_size,
                as_df=False,
            )
            if q.features:
                features.extend(q.features)
            if len(q.features) < page_size:
                break
        if not features:
            raise Exception("No features with pagination")
    except Exception as e:
        # Si falla la paginación, intenta descargar todo de una sola vez
        try:
            q = lyr.query(where="1=1", out_fields="*", return_geometry=True, as_df=False)
            features = q.features if q.features else []
        except Exception as e2:
            print(f"    ⚠️ Error al descargar {layer_name}: {e2}")
            return

    if features:
        fs = FeatureSet(features)
        geojson = fs.to_geojson
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(geojson)
        print(f"    ✅ Guardado completo: {fname} ({len(features)} features)")
    else:
        print(f"    ⚠️ Sin datos para {layer_name}")

## Runtime

In [ ]:
def main():
    gis = GIS()  # acceso anónimo
    services = get_all_mapservers()
    print(f"🧭 MapServers encontrados: {len(services)}")
    for service_url in services:
        service_name = service_url.split("/")[-2]
        print(f"\n📦 Servicio: {service_name}")
        try:
            layers = get_layers(service_url)
            for layer in layers:
                print(f"  🔍 Capa {layer['id']}: {layer['name']}")
                download_layer_arcgis(service_url, layer, gis)
        except Exception as e:
            print(f"  ⚠️ Error en servicio {service_name}: {e}")
    print("\n🏁 Proceso finalizado.")

main()

# Tests